In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("logistic-regression").getOrCreate()

21/12/15 03:47:13 WARN Utils: Your hostname, 6miniui-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.200.112 instead (on interface en0)
21/12/15 03:47:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/15 03:47:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/15 03:47:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/15 03:47:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/15 03:47:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/12/15 03:47:15 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
# 파이프라인에 포커스를 맞춘다.
# 파이프라인은 데이터의 여러가지 처리 단계를 거칠 때 유용하게 쓰인다.
# 대표적인 예가 텍스트 테이터로서, 토크나이징이나 글자의 수를 센다던지 원핫 인코딩을 하는 등의 여러가지 처리 방법이 있다.
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [3]:
# 예제 데이터 생성
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [4]:
# 글자를 스플릿하기 위한 토크나이저와 해싱 TF 생성
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [5]:
lr = LogisticRegression(maxIter=30, regParam=0.001)

In [6]:
# 파이프라인 생성
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [7]:
model = pipeline.fit(training)

21/12/15 03:54:06 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
21/12/15 03:54:06 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
21/12/15 03:54:07 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/12/15 03:54:07 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [8]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [9]:
prediction = model.transform(test)

In [10]:
prediction.select(["id", "text", "probability", "prediction"]).show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.63102699631690...|       0.0|
|  5|             l m n|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[0.13563147748816...|       1.0|
|  7|     apache hadoop|[0.99563405823116...|       0.0|
+---+------------------+--------------------+----------+

